# Openpose 1.7.0 Demo -- I did not author this notebook


In [ ]:
!nvidia-smi

Fri May 19 19:49:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    40W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 最初に、ランタイムの種類を「GPU」に変更してください。（ランタイム＞ランタイムのタイプを変更）
# GPUが有効になっていることを確認します。

# First, change the runtime type to "GPU".
# Confirm runtime to GPU
! nvcc --version
! nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Fri May 19 19:49:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    40W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |       

In [ ]:
# Openpose バージョン指定タグ
# Openpose version tag
ver_openpose = "v1.7.0"
! echo $ver_openpose

v1.7.0


In [ ]:
# CMakeが古いとOpenpose（CUDA10)が失敗するので、バージョンを確認します。（BugfixはCMake 3.12.3）
# If CMake is old, Openpose build fails, so download the latest version
# https://developercommunity.visualstudio.com/content/problem/354325/cmake-project-cannot-detect-cuda-10.html
! cmake --version

cmake version 3.25.2

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [ ]:
# 2021.03.03時点の最新CMakeを再ビルド（15分くらい）
# Rebuild the latest CMake as of 2021.03.03 (about 15 minutes)
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.13.4/cmake-3.13.4.tar.gz"
! tar xf cmake-3.13.4.tar.gz
! cd cmake-3.13.4 && ./configure && make && sudo make install

Streaming output truncated to the last 5000 lines.
-- Looking for connect in socket;dl - not found
-- Looking for gethostbyname in c
-- Looking for gethostbyname in c - found
-- Looking for recv in network;dl
-- Looking for recv in network;dl - not found
-- Looking for gethostname
-- Looking for gethostname - found
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found version "1.1.1f")  
-- Looking for openssl/crypto.h
-- Looking for openssl/crypto.h - found
-- Looking for openssl/err.h
-- Looking for openssl/err.h - found
-- Looking for openssl/pem.h
-- Looking for openssl/pem.h - found
-- Looking for openssl/rsa.h
-- Looking for openssl/rsa.h - found
-- Looking for openssl/ssl.h
-- Looking for openssl/ssl.h - found
-- Looking for openssl/x509.h
-- Looking for openssl/x509.h - found
-- Looking for openssl/rand.h
-- Looking for openssl/rand.h - found
-- Looking for RAND_status
-- Looking for RAND_status - not found
-- Looking for RAND_screen
-- Looking for RAND_screen - not f

In [ ]:
# ライブラリのインストール

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python2 libs
! sudo apt-get --assume-yes install python-setuptools python-dev build-essential
! sudo easy_install pip
! sudo -H pip install --upgrade numpy protobuf opencv-python
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCV 2.4 -> Added as option
# # sudo apt-get --assume-yes install libopencv-dev
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

#  Openpose の clone
#! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 
! git clone  --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git     

#  Openpose の モデルデータDL
! cd openpose/models && ./getModels.sh

# Openpose の ビルド
! sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
! cd openpose && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,699 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,344 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,175 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,240 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,049 kB]
Hit:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.ne

##### Run on actual videos


In [ ]:
import os
# os.makedirs('/content/openpose/data')

z = ["front_tuck", "back_tuck", "sheep_jump", "split_jump", "full_turn", "l_turn", "back_handspring_so", "back_layout_so", "front_aerial", "side_aerial"]

for u in z:
  os.makedirs(f'/content/openpose/data/{u}')
  os.makedirs(f'/content/openpose/output/{u}')



In [ ]:
import os 
nums = range(1, 101)

for c in z:
  for n in nums:
    className = c
    fileName = className + '_' + str(n) + '.mp4'
    fileName_out = className + '_' + str(n) + '_out.mp4'
    num = str(n)
    out_path = f'/content/openpose/out_json/{className}/{num}'
    os.makedirs(out_path)
    ! cd openpose && ./build/examples/openpose/openpose.bin --video data/$className/$fileName --display 0 --number_people_max 1 --write_keypoint $out_path --write_video ./output/$className/$fileName_out 

Streaming output truncated to the last 5000 lines.
[libx264 @ 0x564e9650d940] i4 v,h,dc,ddl,ddr,vr,hd,vl,hu: 27% 23% 12%  6%  6%  7%  6%  7%  6%
[libx264 @ 0x564e9650d940] i8c dc,h,v,p: 40% 31% 22%  7%
[libx264 @ 0x564e9650d940] Weighted P-Frames: Y:0.0% UV:0.0%
[libx264 @ 0x564e9650d940] ref P L0: 75.6% 10.0% 11.1%  3.3%
[libx264 @ 0x564e9650d940] ref B L0: 88.7% 10.0%  1.3%
[libx264 @ 0x564e9650d940] ref B L1: 95.8%  4.2%
[libx264 @ 0x564e9650d940] kb/s:1405.91
Video saved and temporary image folder removed.
Starting OpenPose demo...
Configuring OpenPose...
Flag `write_keypoint` is deprecated and will eventually be removed. Please, use `write_json` instead.
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/l

In [ ]:
 ! cd openpose && ./build/examples/openpose/openpose.bin --video data/split_jumop --display 0 --number_people_max 1 --write_keypoint /content/openpose/out_json/ --write_video ./output/ 

Starting OpenPose demo...
Configuring OpenPose...
Flag `write_keypoint` is deprecated and will eventually be removed. Please, use `write_json` instead.
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.

(openpose.bin:33509): GStreamer-CRITICAL **: 16:28:20.494: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:1] global ../modules/videoio/src/cap_gstreamer.cpp (1500) open OpenCV | GStreamer warning: OpenCV backend does not support this file type (extension): ./output/
[ WARN:1] global ../modules/videoio/src/cap_gstreamer.cpp (1274) close_ OpenCV | GStreamer warning: No source in GStreamer pipeline. Ignore
[ERROR:1] global ../modules/videoio/src/cap.cpp (415) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.2.0) ../modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): ./output/ in function 'icvExtractPattern'



Erro

##### Download json 

In [ ]:
from google.colab import files

!zip -r /content/sheep_jump_json.zip /content/openpose/out_json/sheep_jump

files.download('/content/sheep_jump_json.zip')

Streaming output truncated to the last 5000 lines.
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000025_pose.yml (deflated 58%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000019_pose.yml (deflated 59%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000001_pose.yml (deflated 58%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000045_pose.yml (deflated 57%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000030_pose.yml (deflated 58%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000031_pose.yml (deflated 59%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000016_pose.yml (deflated 58%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000027_pose.yml (deflated 58%)
  adding: content/openpose/out_json/sheep_jump/62/sheep_jump_62_000000000010_pose.yml (deflated 58%)
  adding: content/openpose/out_json/shee

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### Download Videos 

In [ ]:
!zip -r /content/openpose/front_tuck_videos.zip /content/openpose/output/front_tuck

files.download('/content/openpose/front_tuck_videos.zip')

  adding: content/openpose/output/front_tuck/ (stored 0%)
  adding: content/openpose/output/front_tuck/front_tuck_93_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_99_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_49_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_91_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_25_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_94_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_97_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_92_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_55_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_43_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_tuck_88_out.mp4 (deflated 0%)
  adding: content/openpose/output/front_tuck/front_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # サンプルの実行確認
## For testing purposes only 
# ! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/back_layout_so_1.mp4 --display 0 --number_people_max 1 --write_video ./back_layout_so_1_out.mp4
# # ! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/back_layout_so_1.mp4 --display 0 --render_pose 0 --number_people_max 1 --write_keypoint ./out_json/  #--write_video ./back_layout_so_1_out.mp4

# # ! cd openpose && ./build/examples/openpose/openpose.bin --video ../../data/split_jump_1.mp4  --display 0 --write_coco_json output_json/ #--render_pose 0 --write_keypoint ./out_json/ # --write_video ./output/sheep_jump_1_out.avi
# # !watch -n 1.0 nvidia-smi
# # ! cd openpose && ./build/examples/openpose/openpose.bin --image_dir examples/media/ --display 0 --write_images ./output_images/
# !nvidia-smi


Starting OpenPose demo...
Configuring OpenPose...
Flag `write_keypoint` is deprecated and will eventually be removed. Please, use `write_json` instead.
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --e